In [55]:
import re
import csv
import pandas as pd
import numpy as np

In [56]:
def read_input_file(file_name):

    feature_vectors = []
    file = open(file_name,'r')

    for line in file:
        vectors = []
        data = re.split(r'[()\s]\s*', line)
        while '' in data:
            data.remove('')

        for item in data:
            vectors.append(int(item))
        feature_vectors.append(vectors)
    with open("input.csv","w+") as my_csv:
        csvWriter = csv.writer(my_csv,delimiter=',')
        csvWriter.writerows(feature_vectors)
read_input_file('ClassifiedSetData.txt')

In [ ]:
class MLP:
    def __init__(self):
        self.number_of_nodes=10
        self.max_feature_value=90
        self.num_output_nodes=8
        
    def split_data(self):
        self.df=pd.read_csv('input.csv',header=None)
        self.df=self.df.rename({0:'ID',1:'Feature_1',2:'Feature_2',3:'Feature_3',4:'Feature_4',5:'Feature_5',6:'Feature_6',7:'Feature_7',8:'Feature_8',9:'Feature_9',10:'Feature_10',11:'Output'}, axis=1)
        self.train_data, self.validate_data, self.test_data = np.split(self.df.sample(frac=1,random_state=42), [int(.6*len(self.df)), int(.8*len(self.df))])
    def multiplication_between_layers(self,weights, inputs):
        total=0
        for i in range(len(weights)):
            total += (inputs[i] * weights[i])
        return total   
    
    def logistic(self, x):
        return 1 / ( 1 + math.exp(-x))
    
    def normalize_array(self, features,normalize_value):
        normalized_array=[]
        for i in features:
            normalized_array.append(i/normalize_value)
        return normalized_array
    
    def initialize_weight(self,x_dim,y_dim):
        return np.random.randn(x_dim,y_dim)*np.sqrt(1/y_dim)
    
    def set_weights(self,Wi,Wh1,Wh2,Wh3,Wh4,Wh5,Wh6):
        self.Wi=Wi
        self.Wh1=Wh1
        self.Wh2=Wh2
        self.Wh3=Wh3
        self.Wh4=Wh4
        self.Wh5=Wh5
        self.Wh6=Wh6
        
    def forward_propagation(self,training_data):
            
        #First_Hidden_Layer
        self.hidden_layer_1 = []
        for i in range(self.number_of_nodes):
            self.hidden_layer_1.append(self.logistic(self.multiplication_between_layers(self.Wi[i], training_data)))


        #Second_Hidden_Layer
        self.hidden_layer_2 = []
        for i in range(self.number_of_nodes):
            self.hidden_layer_2.append(self.logistic(self.multiplication_between_layers(self.Wh1[i], self.hidden_layer_1)))


        #Third_Hidden_Layer
        self.hidden_layer_3 = []
        for i in range(self.number_of_nodes):
            self.hidden_layer_3.append(self.logistic(self.multiplication_between_layers(self.Wh2[i],  self.hidden_layer_2)))

        #Fourth_Hidden_Layer
        self.hidden_layer_4 = []
        for i in range(self.number_of_nodes):
            self.hidden_layer_4.append(self.logistic(self.multiplication_between_layers(self.Wh3[i],  self.hidden_layer_3)))

        #Fifth_Hidden_Layer
        self.hidden_layer_5 = []
        for i in range(self.number_of_nodes):
            self.hidden_layer_5.append(self.logistic(self.multiplication_between_layers(self.Wh4[i],  self.hidden_layer_4)))

            
        #Sixth_Hidden_Layer
        self.hidden_layer_6 = []
        for i in range(self.number_of_nodes):
            self.hidden_layer_6.append(self.logistic(self.multiplication_between_layers(self.Wh5[i],  self.hidden_layer_5)))
    
        self.output_layer = []
    
        for i in range(self.num_output_nodes):
            self.output_layer.append(self.logistic(self.multiplication_between_layers(self.Wh6[i], self.hidden_layer_6)))
        return self.hidden_layer_6, self.output_layer
 
    def back_propagation(self,learning_rate, min_accuracy):
        epochs = 0
        while self.accuracy(self.train_data) < min_accuracy:
            epochs += 1
            for value in self.train_data.values:
                training_vec = self.normalize_array(list(value)[1:11], self.max_feature_value)
                self.forward_propagation(training_vec)
   


                # Calculate errors
                delta = [] 
                for j in range(self.num_output_nodes):
                    y = self.output_layer[j]
                    t = 0.2
                    if j == list(value)[-1]:
                        t = 0.8
                    delta.append(y * (1 - y) * (t - y))
                    
                    

                delta1 = [] 
                for j in range(self.number_of_nodes):
                    h = self.hidden_layer_6[j]
                    s = self.multiplication_between_layers(self.Wh6[:,j], delta)
                    delta1.append(h * (1 - h) * s)
                    
                    


                delta2 = [] 
                for j in range(self.number_of_nodes):
                    h = self.hidden_layer_5[j]
                    s = self.multiplication_between_layers(self.Wh5[:,j], delta1)
                    delta2.append(h * (1 - h) * s)
                    

                delta3 = [] 
                for j in range(self.number_of_nodes):
                    h = self.hidden_layer_4[j]
                    s = self.multiplication_between_layers(self.Wh4[:,j], delta2)
                    delta3.append(h * (1 - h) * s)
                    

                delta4 = [] 
                for j in range(self.number_of_nodes):
                    h = self.hidden_layer_3[j]
                    s = self.multiplication_between_layers(self.Wh3[:,j], delta3)
                    delta4.append(h * (1 - h) * s)
                    

                delta5 = [] 
                for j in range(self.number_of_nodes):
                    h = self.hidden_layer_2[j]
                    s = self.multiplication_between_layers(self.Wh2[:,j], delta4)
                    delta5.append(h * (1 - h) * s)
                    
                    
                delta6 = [] 
                for j in range(self.number_of_nodes):
                    h = self.hidden_layer_1[j]
                    s = self.multiplication_between_layers(self.Wh1[:,j], delta5)
                    delta6.append(h * (1 - h) * s)

                    

                # Update Weights
                for j in range(self.number_of_nodes):
                    for i in range(self.num_output_nodes):
                        self.Wh6[i][j] = self.Wh6[i][j] + learning_rate * delta[i] * self.hidden_layer_6[j]
                    for k in range(self.number_of_nodes):
                        self.Wh5[j][k] = self.Wh5[j][k] + learning_rate * delta1[j] * self.hidden_layer_5[k]
                    for k in range(self.number_of_nodes):
                        self.Wh4[j][k] = self.Wh4[j][k] + learning_rate * delta2[j] * self.hidden_layer_4[k]
                    for k in range(self.number_of_nodes):
                        self.Wh3[j][k] = self.Wh3[j][k] + learning_rate * delta3[j] * self.hidden_layer_3[k]
                    for k in range(self.number_of_nodes):
                        self.Wh2[j][k] = self.Wh2[j][k] + learning_rate * delta4[j] * self.hidden_layer_2[k]
                    for k in range(self.number_of_nodes):
                        self.Wh1[j][k] = self.Wh1[j][k] + learning_rate * delta5[j] * self.hidden_layer_1[k]
                    for k in range(self.number_of_nodes):
                        self.Wi[j][k] = self.Wi[j][k]   + learning_rate * delta6[j] * training_vec[k]
                        

                

                              
        return epochs
    def accuracy(self, input_data):
        correct = 0
        for data in input_data.values:
            hl, ol = self.forward_propagation(self.normalize_array(list(data)[1:11], self.max_feature_value))
            if (np.argmax(ol) == list(data)[-1]):
                correct += 1
        return correct / len(input_data


        

d=MLP()
d.split_data()
Weights_input_layer=d.initialize_weight(10,10)
Weights_hidden_layer_1=d.initialize_weight(10,10)
Weights_hidden_layer_2=d.initialize_weight(10,10)
Weights_hidden_layer_3=d.initialize_weight(10,10)
Weights_hidden_layer_4=d.initialize_weight(10,10)
Weights_hidden_layer_5=d.initialize_weight(10,10)
Weights_hidden_layer_6=d.initialize_weight(8,10)
d.set_weights(Weights_input_layer,Weights_hidden_layer_1,Weights_hidden_layer_2,Weights_hidden_layer_3,Weights_hidden_layer_4,Weights_hidden_layer_5,Weights_hidden_layer_6)
print(d.back_propagation(0.1, 0.8))
# print("Holdout Accuracy: " + str(accuracy(Holdout_Set)))
# print("Training Accuracy: " + str(accuracy(Training_Set)))